In [1]:
import pandas as pd
import numpy as np
np.random.seed(11172613)

In [2]:
data_pickles = ['./data2015.pkl', './data2016.pkl','./data2017.pkl', './data2018.pkl','./data2019.pkl', './data2020.pkl']
dfs = [pd.read_pickle(f) for f in data_pickles]
df = pd.concat(dfs)
df

,quoteID,quotation,speaker,date,numOccurrences
5265,2015-08-19-056730,It's very important to understand how the ocea...,Helen McGregor,2015-08-19 11:46:00,2
5891,2015-09-28-067347,man-made emissions are an insignificant part o...,John Abraham,2015-09-28 23:16:52,1
7868,2015-09-29-052783,In the final months before crucial climate tal...,Bill de Blasio,2015-09-29 16:10:01,1
9207,2015-10-29-104475,The Baker Polito Administration is fully devot...,Peter Lorenz,2015-10-29 10:49:41,3
9709,2015-05-07-042445,The general narrative is addressing climate ch...,Dallas Burtraw,2015-05-07 17:55:43,2
...,...,...,...,...,...
5240963,2020-02-06-089077,The expected increase in severe weather due to...,Deb Gardner,2020-02-06 22:38:10,1
5241586,2020-02-05-114487,We will be an energetic champion of free trade...,Dominic Raab,2020-02-05 16:30:31,2
5243208,2020-02-10-057505,"Obama got health care, Trump got his tax cut, ...",Tom Steyer,2020-02-10 11:00:08,1
5243271,2020-02-19-061285,Our colleagues will also continue to work toge...,Markus Dohle,2020-02-19 08:37:21,1


In [3]:
docs = df['quotation'].to_numpy()

In [4]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

docs = [[token for token in doc if not token in stopwords ] for doc in docs]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamchellaoui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [7]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [8]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=1)

In [9]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [10]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 8666
Number of documents: 130752


In [14]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = 1  # Don't evaluate model perplexity, takes too much time.

    # Make a index to word dictionary.
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token
    
    for num_topics in range(start, limit, step):
        print("Current pass {}".format(num_topics))
        model =  models.ldamodel.LdaModel(
            corpus=corpus,
            id2word=id2word,
            num_topics=num_topics,
        )        
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [12]:
from gensim import models

In [ ]:
import gensim
from gensim.models import CoherenceModel
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=docs, start=2, limit=40, step=6)

In [13]:
# Show graph
import matplotlib.pyplot as plt
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()


NameError: name 'coherence_values' is not defined

In [ ]:
# Train LDA model.
from gensim.models import LdaModel
np.random.seed(6080)
# Set training parameters.
num_topics = 8
chunksize = 2000
passes = 20
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [14]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
top_topics

Average topic coherence: -3.4830.


[([(0.11989736, 'climate'),
   (0.103969425, 'change'),
   (0.0136989765, 'people'),
   (0.013464996, 'crisis'),
   (0.013036855, 'need'),
   (0.011228675, 'world'),
   (0.009658497, 'u'),
   (0.009634291, 'issue'),
   (0.009474469, 'action'),
   (0.008432224, 'one'),
   (0.008079792, 'year'),
   (0.007928894, 'time'),
   (0.007893851, 'impact'),
   (0.0076387897, 'like'),
   (0.007513799, 'way'),
   (0.0074681407, 'make'),
   (0.0068873516, 'going'),
   (0.0064701573, 'think'),
   (0.006330662, 'take'),
   (0.0061290557, 'future')],
  -2.624541338164764),
 ([(0.021694943, 'new'),
   (0.019999675, 'energy'),
   (0.015101945, 'community'),
   (0.014324759, 'work'),
   (0.014250023, 'help'),
   (0.014166249, 'government'),
   (0.01222434, 'economy'),
   (0.011066337, 'health'),
   (0.010574554, 'environmental'),
   (0.010133865, 'environment'),
   (0.009880448, 'fight'),
   (0.00911218, 'support'),
   (0.00893478, 'solution'),
   (0.008837341, 'plan'),
   (0.008619852, 'business'),
   (0

In [15]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [16]:
viz_data = gensimvis.prepare(model, corpus, dictionary)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
pyLDAvis.display(viz_data)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.save_html(viz_data, 'LDAvis.html')

In [18]:
classes = []
for i in range(len(df)):
    classes.append(max(model.get_document_topics(corpus[i]), key=lambda t: t[1])[0])

df['class'] = classes


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
df.head()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,quoteID,quotation,speaker,date,numOccurrences,class
5265,2015-08-19-056730,"[it, s, very, important, to, understand, how, ...",Helen McGregor,2015-08-19 11:46:00,2,2
5891,2015-09-28-067347,"[man, made, emissions, are, an, insignificant,...",John Abraham,2015-09-28 23:16:52,1,7
7868,2015-09-29-052783,"[in, the, final, months, before, crucial, clim...",Bill de Blasio,2015-09-29 16:10:01,1,7
9207,2015-10-29-104475,"[the, baker, polito, administration, is, fully...",Peter Lorenz,2015-10-29 10:49:41,3,3
9709,2015-05-07-042445,"[the, general, narrative, is, addressing, clim...",Dallas Burtraw,2015-05-07 17:55:43,2,7


In [19]:
speakers_per_class = pd.DataFrame(df.groupby('class')['speaker'].value_counts())
speakers_per_class = speakers_per_class.rename({'speaker':'count'}, axis='columns')

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
speakers_per_class.reset_index()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,class,speaker,count
0,0,Harsh Vardhan,4
1,0,Prakash Javadekar,4
2,0,José Graziano da Silva,3
3,0,Mohammad Saleem,3
4,0,Antonio Guterres,2
...,...,...,...
33418,7,Ángel Cabrera,1
33419,7,Åsmund Asdal,1
33420,7,Éanna Ni Lamhna,1
33421,7,Örjan Bodin,1


In [21]:
speakers_per_class.groupby('class').head()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


count
class speaker                      
0     Harsh Vardhan               4
      Prakash Javadekar           4
      José Graziano da Silva      3
      Mohammad Saleem             3
      Antonio Guterres            2
1     Donald Trump                4
      Gary Bauer                  2
      James Shaw                  2
      Dingdong Dantes             1
      Francis Pangilinan          1
2     Scott Pruitt               61
      Pope Francis               17
      Ted Cruz                   15
      Will Steffen               11
      Donald Trump                9
3     Catherine McKenna          67
      Andrew Cuomo               53
      Gina McCarthy              53
      Narendra Modi              47
      Charlie Baker              43
4     Michel Jarraud             14
      Petteri Taalas             13
      Dave Allen                 11
      Michael Mann                9
      Ashraf Fawzy                8
5     Bernie Sanders             39
      Scott Pruitt               18
      Hillary Clinton            16
      Marc Morano                15
      Carly Fiorina              14
6     Andrew Wheeler             31
      Scott Pruitt               23
      Brent Erickson             14
      Brian Jennings             11
      Charlie Baker              11
7     Bernie Sanders           1225
      President Barack Obama    877
      Narendra Modi             721
      Jay Inslee                686
      Michael Mann              655

In [22]:
speakers_to_save = speakers_per_class.groupby('class')

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
speakers_per_class.to_pickle("speaker_per_class.pkl")

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
df.to_pickle("quotes_topic_associated.pkl")

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
